### 1、	掌握常量张量的初始化，运算，变量的赋值，加减乘除等

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as np
import matplotlib.pyplot as plt
import sklearn
import os,sys
%matplotlib inline

In [20]:
# tf常量的运算
t = tf.constant([[1.,2,3],[4,5,6]])
print(t[:,2]) # 取第二列元素
print('-'*30)
print(t[...,1])
a = t.numpy()
print(type(a))
print('-'*30)
b = tf.constant(a)
print(type(b))
print(t+10)
print('-'*30)
print(t @ tf.transpose(t)) # @表示矩阵乘法
print('-'*30)
print(tf.square(t))

tf.Tensor([3. 6.], shape=(2,), dtype=float32)
------------------------------
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
<class 'numpy.ndarray'>
------------------------------
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
------------------------------
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)
------------------------------
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)


In [38]:
# tf变量的运算
t = tf.Variable([[1.,2,3],[4,5,6]])
t[:1].assign(t[:1]*10) # assign是对变量进行原地的修改，提升了效率
print(t)
print('-'*30)
t[:,2].assign([13,16])
print(t)
print('-'*30)
t[1,1].assign(200)
print(t)
print('-'*30)
t.scatter_nd_update(indices=[[0,0],[1,2]],updates=[500,600])
print(t)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[10., 20., 30.],
       [ 4.,  5.,  6.]], dtype=float32)>
------------------------------
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[10., 20., 13.],
       [ 4.,  5., 16.]], dtype=float32)>
------------------------------
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 10.,  20.,  13.],
       [  4., 200.,  16.]], dtype=float32)>
------------------------------
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[500.,  20.,  13.],
       [  4., 200., 600.]], dtype=float32)>


### 2、	掌握RaggedTensor和SparseTensor的初始化 ，以及与tensor之间的互相转换

In [46]:
# ragged tensor
r1 = tf.ragged.constant([[1.,2,3],[4],[],[6,7]]) # 不规则矩阵
r2 = tf.ragged.constant([[4.,5,6],[100,200],[],[]])
print(r1)
print('-'*30)
print(tf.concat([r1,r2],axis=0))
print('-'*30)
print(tf.concat([r1,r2],axis=1))
print(type(r1.to_tensor()))  #ragged tensor --> tensor

<tf.RaggedTensor [[1.0, 2.0, 3.0], [4.0], [], [6.0, 7.0]]>
------------------------------
<tf.RaggedTensor [[1.0, 2.0, 3.0], [4.0], [], [6.0, 7.0], [4.0, 5.0, 6.0], [100.0, 200.0],
 [], []]>
------------------------------
<tf.RaggedTensor [[1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [4.0, 100.0, 200.0], [], [6.0, 7.0]]>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [52]:
# sparse tensor
s = tf.SparseTensor(indices=[[0,0],[1,1],[2,2],[3,3]],
                    values = [4.,5,6,7],
                    dense_shape=[4,4])
print(s)
t = tf.sparse.to_dense(s)  # sparse --> tensor
print('-'*30)
print(t)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 1]
 [2 2]
 [3 3]], shape=(4, 2), dtype=int64), values=tf.Tensor([4. 5. 6. 7.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))
------------------------------
tf.Tensor(
[[4. 0. 0. 0.]
 [0. 5. 0. 0.]
 [0. 0. 6. 0.]
 [0. 0. 0. 7.]], shape=(4, 4), dtype=float32)


### 3、	掌握自定义损失函数，自定义层

In [77]:
# 自定义损失函数
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
house_price = fetch_california_housing()
x_train,x_test,y_train,y_test = train_test_split(
    house_price.data,house_price.target,random_state=11)
x_train,x_val,y_train,y_val = train_test_split(
    x_train,y_train,random_state=22)

In [78]:
print(x_train.shape)
print(x_test.shape)

(11610, 8)
(5160, 8)


In [79]:
from sklearn.preprocessing import StandardScaler
stand = StandardScaler()
x_train = stand.fit_transform(x_train)
x_val = stand.transform(x_val)
x_test = stand.transform(x_test)

In [85]:
def customize_mse(y_true,y_pred):
    loss = tf.reduce_mean(tf.square(y_pred-y_true))
    return loss
model = keras.models.Sequential(
    [layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
     layers.Dense(1)])
model.compile(loss=customize_mse,
              metrics=['mean_squared_error'],
              optimizer='sgd')

In [86]:
history = model.fit(x_train,y_train,
                    validation_data=(x_val,y_val),
                    epochs=20)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 0.8550 - mean_squared_error: 0.8550 - val_loss: 0.5832 - val_mean_squared_error: 0.5832
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5086 - mean_squared_error: 0.5086 - val_loss: 0.4978 - val_mean_squared_error: 0.4978
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4596 - mean_squared_error: 0.4596 - val_loss: 0.4686 - val_mean_squared_error: 0.4686
Epoch 4/20
363/363 [==============================] - 0s 998us/step - loss: 0.4393 - mean_squared_error: 0.4393 - val_loss: 0.4609 - val_mean_squared_error: 0.4609
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4285 - mean_squared_error: 0.4285 - val_loss: 0.4568 - val_mean_squared_error: 0.4568
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4283 - mean_squared_error: 0.4283 - val_loss: 0.4422 - val_mean_squared_error: 0.4422
Epoch 7/20
363/363 [======

In [92]:
# 自定义层
class CustomizeLayer(keras.layers.Layer):
    def __init__(self,units,activation=None,**kwargs):
        self.units = units
        self.activation=keras.layers.Activation(activation)
        super(CustomizeLayer, self).__init__(**kwargs)

    def build(self,input_shape):
        """构建层"""
        self.kernel=self.add_weight(name='kernel',
                                    shape=(input_shape[1],self.units),
                                    initializer='uniform',
                                    trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.units),
                                    initializer='zeros',
                                    trainable=True)
        super(CustomizeLayer,self).build(input_shape)

    def call(self,x):
        """前向传播的计算"""
        return self.activation(x @ self.kernel + self.bias)
model = keras.models.Sequential(
    [CustomizeLayer(30,activation='relu',
                    input_shape=x_train.shape[1:]),
     CustomizeLayer(1)]
)

In [93]:
model.compile(loss=customize_mse,
              metrics=['mean_squared_error'],
              optimizer='sgd')

In [94]:
history = model.fit(x_train,y_train,
                    validation_data=(x_val,y_val),
                    epochs=20)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.0755 - mean_squared_error: 1.0755 - val_loss: 0.6331 - val_mean_squared_error: 0.6331
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5484 - mean_squared_error: 0.5484 - val_loss: 0.5496 - val_mean_squared_error: 0.5496
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4914 - mean_squared_error: 0.4914 - val_loss: 0.5221 - val_mean_squared_error: 0.5221
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4707 - mean_squared_error: 0.4707 - val_loss: 0.5116 - val_mean_squared_error: 0.5116
Epoch 5/20
363/363 [==============================] - 0s 950us/step - loss: 0.4603 - mean_squared_error: 0.4603 - val_loss: 0.4934 - val_mean_squared_error: 0.4934
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4521 - mean_squared_error: 0.4521 - val_loss: 0.4799 - val_mean_squared_error: 0.4799
Epoch 7/20
363/363 [======

### 4、	掌握tf.function的使用

In [96]:
def cube(x):
    """python函数"""
    return x**3
print(cube(2))
print(cube(tf.constant(2.0)))

8
tf.Tensor(8.0, shape=(), dtype=float32)


In [97]:
type(cube)

function

In [101]:
# 把python函数转换为tensorflow函数
tf_cube = tf.function(cube)
print(type(tf_cube))
tf_cube(3.)
tf_cube.python_function(3) # 使用python函数方式调用

<class 'tensorflow.python.eager.def_function.Function'>


27